Compares the manual clasification of users with the one obtained from the label propagation processes to check the performance

In [ ]:
import pandas as pd

# nodos
path = "../data/"
file_nodos16 = "neo4j_nodos_16.csv"
file_nodos20 = "neo4j_nodos_20.csv"
file_nodos24 = "neo4j_nodos_24.csv"

years = [16,20,24]
nodos = [file_nodos16,file_nodos20,file_nodos24]
for p,f in enumerate(nodos):
    df = pd.read_csv(f)
    df["year"] = years[p]
    df_nodos = df if p==0 else pd.concat([df_nodos,df])
df_nodos.rename(columns={'tipo': 'party_contagio'}, inplace=True)


# fichero clasificado manualmente

file_manual = "muestra_contagio_completada.csv"

df_manual = pd.read_csv(file_manual)
df_manual.party = df_manual.party.str.upper()
df_manual.rename(columns={'party': 'party_manual'}, inplace=True)

df_manual

In [ ]:
df_nodos

In [ ]:
df_both = df_nodos.merge(df_manual, on=["screen","year"], how="inner")
df_both

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer,f1_score, accuracy_score, recall_score,cohen_kappa_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def show_results(y_real,y_pred):
    print("Accuracy ",round(accuracy_score(y_real,y_pred),4),\
              "\nRecall ",round(recall_score(y_real,y_pred),4),\
              "\nKappa ", round(cohen_kappa_score(y_real,y_pred),4)
         )
    print('\nClassification Report: \n', classification_report(y_real, y_pred))

    k =  cohen_kappa_score(y_real,y_pred)
    print("kappa ",k)
    cm = confusion_matrix(y_real, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.show()


In [ ]:
show_results(df_both.party_manual.replace({"REP":0,"DEM":1}),df_both.party_contagio.replace({"REP":0,"DEM":1}))

In [ ]:
df_discrepancias = df_both[df_both.party_manual != df_both.party_contagio][["year","screen","contagio","party_manual","party_contagio"]]
df_discrepancias

In [ ]:
df_discrepancias.contagio.value_counts(normalize=True)

In [ ]:
df_both.contagio.value_counts(normalize=True)

In [ ]:
filtro = df_both.contagio=="COMMON"
show_results(df_both[filtro].party_manual.replace({"REP":0,"DEM":1}),df_both[filtro].party_contagio.replace({"REP":0,"DEM":1}))

In [ ]:
filtro = df_both.contagio=="SIMILAR"
show_results(df_both[filtro].party_manual.replace({"REP":0,"DEM":1}),df_both[filtro].party_contagio.replace({"REP":0,"DEM":1}))

In [ ]:
filtro = df_both.contagio=="majority"
show_results(df_both[filtro].party_manual.replace({"REP":0,"DEM":1}),df_both[filtro].party_contagio.replace({"REP":0,"DEM":1}))

## STATISTICS

In [ ]:
user_file = "usa162024.csv"
import pandas as pd
df_user = pd.read_csv(user_file).rename(columns={"User":"screen"})
df_user

In [ ]:
filtro = (df_nodos.sent=="manual") | (df_nodos.contagio=="COMMON")
df = df_nodos[filtro].reset_index(drop=True)
df_vota = df.rename(columns={"party_contagio":"vota"})
df_vota

In [ ]:
df_vota.boxplot(column = "nTweets", by="year",  figsize=(15,10))

In [ ]:
df_props = df_user.groupby(['screen', 'year', 'party']).size().unstack(fill_value=0).reset_index()
df_props["total"] = df_props["dem"]+df_props["rep"]
df_props["dem_p"] = df_props["dem"]/df_props["total"]
df_props["rep_p"] = df_props["rep"]/df_props["total"]
df_props

In [ ]:
df_props_completo = df_props.merge(right=df_vota, on=["screen","year"])
df_props_completo

In [ ]:
medias = df_props_completo.groupby(['vota', 'year'])[['dem_p', 'rep_p']].mean().reset_index()
medias

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Filter the data to keep vota == "REP"
medias_rep = medias[medias['vota'] == 'REP']

# Create the graph
fig, ax = plt.subplots(figsize=(8, 6))

# Define the bars position
x = medias_rep['year']
width = 0.35  # Width of bar

# Draw bars
ax.bar(x - width/2, medias_rep['dem_p'], width=width, label='dem_p', color="blue")
ax.bar(x + width/2, medias_rep['rep_p'], width=width, label='rep_p', color="red")

# Add labels and title
ax.set_xlabel('Año')
ax.set_ylabel('Proporción media')
ax.set_title('Proporción media de tweets sobre DEM y REP (votantes REP)')
ax.set_xticks(x)
ax.legend()
plt.grid()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

medias_rep = medias[medias['vota'] == 'DEM']

fig, ax = plt.subplots(figsize=(8, 6))

x = medias_rep['year']
width = 0.35

ax.bar(x - width/2, medias_rep['dem_p'], width=width, label='dem_p', color="blue")
ax.bar(x + width/2, medias_rep['rep_p'], width=width, label='rep_p', color="red")

ax.set_xlabel('Año')
ax.set_ylabel('Proporción media')
ax.set_title('Proporción media de tweets sobre DEM y REP (votantes DEM)')
ax.set_xticks(x)
ax.legend()
plt.grid()

plt.show()

Same thing but aggregating tweets instead of users

In [ ]:
df_all = df_user.merge(right=df_vota, on=["screen","year"])
df_all

In [ ]:
df_props = df_all.groupby([ 'year', 'vota', 'party']).size().unstack(fill_value=0).reset_index()
df_props

In [ ]:
df_props["total"] = df_props["dem"]+df_props["rep"]
df_props["dem_p"] = df_props["dem"]/df_props["total"]
df_props["rep_p"] = df_props["rep"]/df_props["total"]
df_props

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

medias_rep = df_props[df_props['vota'] == 'REP']

fig, ax = plt.subplots(figsize=(8, 6))

x = medias_rep['year']
width = 0.35 

ax.bar(x - width/2, medias_rep['dem_p'], width=width, label='dem_p', color="blue")
ax.bar(x + width/2, medias_rep['rep_p'], width=width, label='rep_p', color="red")

ax.set_xlabel('Año')
ax.set_ylabel('Proporción media')
ax.set_title('Proporción media de tweets sobre DEM y REP (votantes REP)')
ax.set_xticks(x)
ax.legend()
plt.grid()

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

medias_rep = df_props[df_props['vota'] == 'DEM']

fig, ax = plt.subplots(figsize=(8, 6))

x = medias_rep['year']
width = 0.35

ax.bar(x - width/2, medias_rep['dem_p'], width=width, label='dem_p', color="blue")
ax.bar(x + width/2, medias_rep['rep_p'], width=width, label='rep_p', color="red")

ax.set_xlabel('Año')
ax.set_ylabel('Proporción media')
ax.set_title('Proporción media de tweets sobre DEM y REP (votantes DEM)')
ax.set_xticks(x)
ax.legend()
plt.grid()

plt.show()

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

rep = df_vota[df_vota.vota=="REP"]
dem = df_vota[df_vota.vota=="DEM"]
rep.nTweets.mean(), dem.nTweets.mean()

In [ ]:
rep.nTweets.hist()

In [ ]:
import numpy as np
from scipy.stats import mannwhitneyu

grupo1 = rep.nTweets
grupo2 = dem.nTweets

# Mann-Whitney U
stat, p = mannwhitneyu(grupo1, grupo2, alternative='greater') #  'two-sided', greater if med(grupo1)>med(grupo2)

print(f"Estadístico U: {stat}")
print(f"Valor p: {p:.4f}")

Conclusion: republicans tweet more

In [ ]:
years = [16,20,24]
for y in years:
    print(y)
    filtror = rep.year == y
    filtrod = dem.year == y

    grupo1 = rep[filtror].nTweets
    grupo2 = dem[filtrod].nTweets

    # Mann-Whitney U
    stat, p = mannwhitneyu(grupo1, grupo2, alternative='two-sided')
    print(f"Rep: {grupo1.mean()}, dem: {grupo2.mean()}")
    
    print(f"Estadístico U: {stat}")
    print(f"Valor p: {p:.8f}")
    stat, p = mannwhitneyu(grupo1, grupo2, alternative='greater')

    print(f"Estadístico U: {stat}")
    print(f"Valor p: {p:.8f}")
    stat, p = mannwhitneyu(grupo1, grupo2, alternative='less')

    print(f"Estadístico U: {stat}")
    print(f"Valor p: {p:.8f}")

In [ ]:
filtror = rep.year == 16
filtrod = dem.year == 16

import numpy as np
from scipy.stats import mannwhitneyu

grupo1 = rep[filtror].followers
grupo2 = dem[filtrod].followers

stat, p = mannwhitneyu(grupo1, grupo2, alternative='less')

print(f"Rep: {grupo1.mean()}, dem: {grupo2.mean()}")
print(f"Estadístico U: {stat}")
print(f"Valor p: {p:.6f}")

In [ ]:
filtror = rep.year == 16
filtrod = dem.year == 16

import numpy as np
from scipy.stats import mannwhitneyu

grupo1 = rep[filtror].verified
grupo2 = dem[filtrod].verified

stat, p = mannwhitneyu(grupo1, grupo2, alternative='less')

print(f"Rep: {grupo1.mean()}, dem: {grupo2.mean()}")
print(f"Estadístico U: {stat}")
print(f"Valor p: {p:.6f}")

Does the proportion of republicans/democrats per year vary significantly?

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

contingencia = pd.crosstab(df_vota['year'], df_vota['vota'])

print("Tabla de contingencia:")
print(contingencia)

# chi-square
chi2, p, dof, expected = chi2_contingency(contingencia)

print(f"\nValor chi2 = {chi2:.4f}")
print(f"Valor p = {p:.4f}")
print(f"Grados de libertad = {dof}")

Yes, there is a significant variation in the expected values for year and vota, to see in what couple is the variation present we compare the parties for each year

In [ ]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

def compara_proporciones(df, year1, year2, partido='DEM'):
    votos1 = df[(df['year'] == year1) & (df['vota'] == partido)].shape[0]
    votos2 = df[(df['year'] == year2) & (df['vota'] == partido)].shape[0]
    
    total1 = df[df['year'] == year1].shape[0]
    total2 = df[df['year'] == year2].shape[0]
    
    counts = [votos1, votos2]
    nobs = [total1, total2]
    
    stat, pval = proportions_ztest(counts, nobs)
    
    print(f"Comparando {year1} vs {year2} para {partido}:")
    print(f"  Proporción {year1}: {votos1}/{total1} = {votos1/total1:.2f}")
    print(f"  Proporción {year2}: {votos2}/{total2} = {votos2/total2:.2f}")
    print(f"  z = {stat:.4f}, p = {pval:.4f}\n")
    return stat, pval

compara_proporciones(df_vota, 16, 20,"DEM")
compara_proporciones(df_vota, 16, 24,"DEM")
compara_proporciones(df_vota, 20, 24,"DEM")
compara_proporciones(df_vota, 16, 20,"REP")
compara_proporciones(df_vota, 16, 24,"REP")
compara_proporciones(df_vota, 20, 24,"REP")

Another way:

In [ ]:
def compara_dem_rep(df, year1, year2):
    sub = df[df['year'].isin([year1, year2])]
    tabla = pd.crosstab(sub['vota'], sub['year'])
    print(f"\nComparando {year1} vs {year2}")
    print(tabla)
    chi2, p, dof, expected = chi2_contingency(tabla)
    print(f"Chi2 = {chi2:.4f}, p = {p:.4f}")
    return chi2, p

compara_dem_rep(df_vota, 16, 20)
compara_dem_rep(df_vota, 16, 24)
compara_dem_rep(df_vota, 20, 24)

To see the concrete values

In [ ]:
proporciones = df_vota.groupby(['year', 'vota']).size().unstack().fillna(0)
proporciones['total'] = proporciones.sum(axis=1)
proporciones['DEM_ratio'] = proporciones['DEM'] / proporciones['total']
proporciones['REP_ratio'] = proporciones['REP'] / proporciones['total']

print(proporciones[['DEM_ratio', 'REP_ratio']])

##### How has the format of tweets evolved over time?
- Number of emojis
- Number of hashtags
- Number of mentions
- Number of urls

In 2020 and 2016 emojis are codified as real emojis (❤️❤️), in 2024 as text (face with heart-shaped eyes, Purple heart)
We will use the emoji library to convert the text emojis to real emojis

In [ ]:
import emoji
import re
import json

# Function to count real emojis in a tweet
def count_real_emojis(text):
    return len([char for char in text if char in emoji.EMOJI_DATA])

# Function to count emoji descriptions (assuming words like "face with heart-shaped eyes" appear in text)
def count_text_emojis(text):
    # We manually look for the text codification that twitter uses for some of the most popular emojis
    # We add the rest by using a JSON file
    with open("emoji-dict.json", "r", encoding="utf-8") as f:
        data = json.load(f)

    # Extract names
    emoji_names = [entry["name"] for entry in data.values()] + ["Call me hand","Leftwards hand","Rightwards hand","Leftwards pushing hand","Leg","Nail polish","Selfie","Writing hand","Index pointing at the viewer","Reversed hand with middle finger extended","Mechanical arm","Flexed biceps","Rightwards pushing hand","Mechanical leg","Foot","Mouth","Biting lip","Tooth","Tongue","Ear","Ear with hearing aid","Nose","Crown","Red heart","Orange heart","Yellow heart","Green heart","Blue heart","Purple heart","Brown heart","Black heart","White heart","Heart with ribbon","Heart with arrow","Sparkling heart","Growing heart","Beating heart","Revolving hearts","Two hearts","Heart exclamation","Broken heart","Heart on fire","Mending heart","Heart decoration","No entry","Squared sos","Hundred points symbol","Red question mark ornament","White question mark ornament","Musical note","Multiple musical notes","Red circle","Orange circle","Yellow circle","Green circle","Blue circle","Purple circle","Ballot box with check","Brown circle","Black circle","White circle","Red square","Orange square","Yellow square","Green square","Blue square","Purple square","Brown square","Black large square","White large square","Black medium square","Speaker with one sound wave","Speaker with three sound waves","Speaker with cancellation stroke","Cheering megaphone","Public address loudspeaker","Bell","Bell with cancellation stroke","American football","Flag of U.S. Virgin Islands","Earth globe americas","Flag of United States","Grinning face","Smiling face with open mouth","Smiling face with open mouth and smiling eyes","Grinning face with smiling eyes","Smiling face with open mouth and cold sweat","Face with tears of joy","Rolling on the floor laughing","Smiling face with halo","Smiling face with sunglasses","Money-mouth face","Smiling face with open hands","Grimacing face","Pleading face","Face with look of triumph","Pouting face","Face with symbols over mouth","Sleepy face","Crying face","Loudly crying face","Hot face"]

    # Extend the regex pattern
    emoji_words_pattern = r'\b(?:' + '|'.join(emoji_names) + r')\b'

    # Find matches
    matches = re.findall(emoji_words_pattern, text, re.IGNORECASE)

    return len(matches)

In [ ]:
df_16_filtered = df_user[df_user.year==16]
df_20_filtered = df_user[df_user.year==20]
df_24_filtered = df_user[df_user.year==24]

# Calculate mean emoji count per tweet for each dataset
df_16_filtered["emoji_count"] = df_16_filtered["Tweet"].astype(str).apply(count_real_emojis)
df_20_filtered["emoji_count"] = df_20_filtered["Tweet"].astype(str).apply(count_real_emojis)
df_24_filtered["emoji_count"] = df_24_filtered["Tweet"].astype(str).apply(count_text_emojis)

In [ ]:
df_16_filtered["emoji_count"].hist(bins=50)

In [ ]:
import numpy as np
from itertools import combinations

# Función para bootstrap del intervalo de confianza
def bootstrap_diff_mean(grupo1, grupo2, n_iter=10000, ci=95):
    diffs = []
    for _ in range(n_iter):
        sample1 = np.random.choice(grupo1, size=len(grupo1), replace=True)
        sample2 = np.random.choice(grupo2, size=len(grupo2), replace=True)
        diffs.append(np.mean(sample1) - np.mean(sample2))
    lower = np.percentile(diffs, (100 - ci) / 2)
    upper = np.percentile(diffs, 100 - (100 - ci) / 2)
    return lower, upper

# Tus datos
grupo1 = df_16_filtered["emoji_count"].values
grupo2 = df_20_filtered["emoji_count"].values
grupo3 = df_24_filtered["emoji_count"].values

todos = [(16, grupo1), (20, grupo2), (24, grupo3)]

# Comparación por pares
for ((y1, g1), (y2, g2)) in combinations(todos, 2):
    print(f"{y1} vs {y2}")
    print(f"Medias: {y1} = {np.mean(g1):.3f}, {y2} = {np.mean(g2):.3f}")
    
    ic_low, ic_up = bootstrap_diff_mean(g1, g2)
    print(f"IC 95% (media {y1} - media {y2}): [{ic_low:.3f}, {ic_up:.3f}]")
    
    if ic_low > 0:
        print(f"=> La media de {y1} es significativamente mayor que la de {y2}")
    elif ic_up < 0:
        print(f"=> La media de {y1} es significativamente menor que la de {y2}")
    else:
        print("=> No hay diferencia significativa entre las medias")
    
    print("=" * 40)


In [ ]:
def count_hashtags(text):
    return len(re.findall(r'#\b[a-zA-Z]\w*', text))

In [ ]:
# Apply to each DataFrame
df_16_filtered["hashtag_count"] = df_16_filtered["Tweet"].astype(str).apply(count_hashtags)
df_20_filtered["hashtag_count"] = df_20_filtered["Tweet"].astype(str).apply(count_hashtags)
df_24_filtered["hashtag_count"] = df_24_filtered["Tweet"].astype(str).apply(count_hashtags)

df_16_filtered["hashtag_count"].hist(bins=50)

In [ ]:
import numpy as np
from itertools import combinations

# Función to bootstrap
def bootstrap_diff_mean(grupo1, grupo2, n_iter=10000, ci=95):
    diffs = []
    for _ in range(n_iter):
        sample1 = np.random.choice(grupo1, size=len(grupo1), replace=True)
        sample2 = np.random.choice(grupo2, size=len(grupo2), replace=True)
        diffs.append(np.mean(sample1) - np.mean(sample2))
    lower = np.percentile(diffs, (100 - ci) / 2)
    upper = np.percentile(diffs, 100 - (100 - ci) / 2)
    return lower, upper

grupo1 = df_16_filtered["hashtag_count"].values
grupo2 = df_20_filtered["hashtag_count"].values
grupo3 = df_24_filtered["hashtag_count"].values

todos = [(16, grupo1), (20, grupo2), (24, grupo3)]

for ((y1, g1), (y2, g2)) in combinations(todos, 2):
    print(f"{y1} vs {y2}")
    print(f"Medias: {y1} = {np.mean(g1):.3f}, {y2} = {np.mean(g2):.3f}")
    
    ic_low, ic_up = bootstrap_diff_mean(g1, g2)
    print(f"IC 95% (media {y1} - media {y2}): [{ic_low:.3f}, {ic_up:.3f}]")
    
    if ic_low > 0:
        print(f"=> La media de {y1} es significativamente mayor que la de {y2}")
    elif ic_up < 0:
        print(f"=> La media de {y1} es significativamente menor que la de {y2}")
    else:
        print("=> No hay diferencia significativa entre las medias")
    
    print("=" * 40)


In [ ]:
# Function to count mentions (excluding "RT @user:" because in 2016 retweets were saved like that and we don't want to include them in the analysis)
def count_mentions(text, exclude_rt=False):
    if exclude_rt:
        text = re.sub(r'^RT @\w+: ', '', text)  # Remove "RT @user: " at the start
    return len(re.findall(r'@\w+', text))  # Count mentions (@ followed by letters, numbers, _)

# Apply to each DataFrame
df_16_filtered["mention_count"] = df_16_filtered["Tweet"].astype(str).apply(lambda x: count_mentions(x, exclude_rt=True))
df_20_filtered["mention_count"] = df_20_filtered["Tweet"].astype(str).apply(lambda x: count_mentions(x, exclude_rt=False))
df_24_filtered["mention_count"] = df_24_filtered["Tweet"].astype(str).apply(lambda x: count_mentions(x, exclude_rt=False))

In [ ]:
df_16_filtered["mention_count"].hist(bins=50)

In [ ]:
import numpy as np
from itertools import combinations

# Función para bootstrap del intervalo de confianza
def bootstrap_diff_mean(grupo1, grupo2, n_iter=10000, ci=95):
    diffs = []
    for _ in range(n_iter):
        sample1 = np.random.choice(grupo1, size=len(grupo1), replace=True)
        sample2 = np.random.choice(grupo2, size=len(grupo2), replace=True)
        diffs.append(np.mean(sample1) - np.mean(sample2))
    lower = np.percentile(diffs, (100 - ci) / 2)
    upper = np.percentile(diffs, 100 - (100 - ci) / 2)
    return lower, upper

# Tus datos
grupo1 = df_16_filtered["mention_count"].values
grupo2 = df_20_filtered["mention_count"].values
grupo3 = df_24_filtered["mention_count"].values

todos = [(16, grupo1), (20, grupo2), (24, grupo3)]

# Comparación por pares
for ((y1, g1), (y2, g2)) in combinations(todos, 2):
    print(f"{y1} vs {y2}")
    print(f"Medias: {y1} = {np.mean(g1):.3f}, {y2} = {np.mean(g2):.3f}")
    
    ic_low, ic_up = bootstrap_diff_mean(g1, g2)
    print(f"IC 95% (media {y1} - media {y2}): [{ic_low:.3f}, {ic_up:.3f}]")
    
    if ic_low > 0:
        print(f"=> La media de {y1} es significativamente mayor que la de {y2}")
    elif ic_up < 0:
        print(f"=> La media de {y1} es significativamente menor que la de {y2}")
    else:
        print("=> No hay diferencia significativa entre las medias")
    
    print("=" * 40)


In [ ]:
# Function to count URLs starting with "https://"
def count_urls(text):
    return len(re.findall(r'https://\S+', text, re.IGNORECASE))

# Apply to each DataFrame
df_16_filtered["url_count"] = df_16_filtered["Tweet"].astype(str).apply(count_urls)
df_20_filtered["url_count"] = df_20_filtered["Tweet"].astype(str).apply(count_urls)
df_24_filtered["url_count"] = df_24_filtered["Tweet"].astype(str).apply(count_urls)


df_16_filtered["url_count"].hist(bins=50)

In [ ]:
import numpy as np
from itertools import combinations

# Función para bootstrap del intervalo de confianza
def bootstrap_diff_mean(grupo1, grupo2, n_iter=10000, ci=95):
    diffs = []
    for _ in range(n_iter):
        sample1 = np.random.choice(grupo1, size=len(grupo1), replace=True)
        sample2 = np.random.choice(grupo2, size=len(grupo2), replace=True)
        diffs.append(np.mean(sample1) - np.mean(sample2))
    lower = np.percentile(diffs, (100 - ci) / 2)
    upper = np.percentile(diffs, 100 - (100 - ci) / 2)
    return lower, upper

# Tus datos
grupo1 = df_16_filtered["url_count"].values
grupo2 = df_20_filtered["url_count"].values
grupo3 = df_24_filtered["url_count"].values

todos = [(16, grupo1), (20, grupo2), (24, grupo3)]

# Comparación por pares
for ((y1, g1), (y2, g2)) in combinations(todos, 2):
    print(f"{y1} vs {y2}")
    print(f"Medias: {y1} = {np.mean(g1):.3f}, {y2} = {np.mean(g2):.3f}")
    
    ic_low, ic_up = bootstrap_diff_mean(g1, g2)
    print(f"IC 95% (media {y1} - media {y2}): [{ic_low:.3f}, {ic_up:.3f}]")
    
    if ic_low > 0:
        print(f"=> La media de {y1} es significativamente mayor que la de {y2}")
    elif ic_up < 0:
        print(f"=> La media de {y1} es significativamente menor que la de {y2}")
    else:
        print("=> No hay diferencia significativa entre las medias")
    
    print("=" * 40)
